In [1]:
# Data Manipulation
import pandas as pd
import numpy as np
# Data Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Time-series
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor

In [2]:
# 1. Data Loading

url = "https://raw.githubusercontent.com/065010-AmanMalhi/adani-energy-demand-forecasting/refs/heads/main/data/Predicting%20Energy%20Consumption.docx-EmbeddedFile.xlsm%20-%20Energy%20prediction%20.csv"

df_raw = pd.read_csv(url)
df = df_raw.copy()

df.head()

,Start time UTC,End time UTC,Electricity consumption (MWh)
0,12/31/15 21:00,12/31/15 22:00,10800
1,12/31/15 22:00,12/31/15 23:00,10431
2,12/31/15 23:00,1/1/16 0:00,10005
3,1/1/16 0:00,1/1/16 1:00,9722
4,1/1/16 1:00,1/1/16 2:00,9599


In [3]:
df.shape

(52966, 3)

In [4]:
# Check for missing values in each column
df.isnull().sum()

,0
Start time UTC,0
End time UTC,0
Electricity consumption (MWh),0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52966 entries, 0 to 52965
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Start time UTC                 52966 non-null  object
 1   End time UTC                   52966 non-null  object
 2   Electricity consumption (MWh)  52966 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [6]:
# 2. Data Preprocessing

# Convert 'Start time UTC' column from string to datetime format
df['Start time UTC'] = pd.to_datetime(df['Start time UTC'])
df['End time UTC'] = pd.to_datetime(df['End time UTC'])

# Sort dataset chronologically
df = df.sort_values('Start time UTC')

# Reset index after sorting to maintain clean sequential indexing
df = df.reset_index(drop=True)

# Check updated data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52966 entries, 0 to 52965
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Start time UTC                 52966 non-null  datetime64[ns]
 1   End time UTC                   52966 non-null  datetime64[ns]
 2   Electricity consumption (MWh)  52966 non-null  int64         
dtypes: datetime64[ns](2), int64(1)
memory usage: 1.2 MB


In [7]:
# Calculate difference between consecutive timestamps
df['time_diff'] = df['Start time UTC'].diff()

# Check frequency of time differences
df['time_diff'].value_counts().head()

,count
time_diff,
0 days 01:00:00,51718
0 days 00:55:00,513
0 days 00:05:00,377
0 days 01:05:00,156
0 days 00:50:00,32


In [8]:
# Observation:
# Majority of intervals are 1 hour.
# Minor irregularities (55 min, 65 min) likely due to daylight saving adjustments.
# Since these reflect real-world operational data, we retain the original timestamps.

# Remove temporary column
df = df.drop(columns=['time_diff'])

In [9]:
# 3. Feature Engineering

# Extract basic time-based features
df['hour'] = df['Start time UTC'].dt.hour
df['day_of_week'] = df['Start time UTC'].dt.dayofweek
df['month'] = df['Start time UTC'].dt.month
df['year'] = df['Start time UTC'].dt.year
df['quarter'] = df['Start time UTC'].dt.quarter

# Create weekend indicator (1 if Saturday/Sunday, else 0)
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

In [10]:
# Lag features: capture past consumption values

# Previous hour consumption (short-term memory)
df['lag_1'] = df['Electricity consumption (MWh)'].shift(1)

# Same hour yesterday (daily pattern)
df['lag_24'] = df['Electricity consumption (MWh)'].shift(24)

# Same hour last week (weekly pattern)
df['lag_168'] = df['Electricity consumption (MWh)'].shift(168)

In [11]:
# Rolling averages: capture recent trend (smoothed signal)

# 24-hour rolling average (previous day's average)
df['rolling_24_mean'] = (
    df['Electricity consumption (MWh)']
    .shift(1)                # shift first to avoid leakage
    .rolling(24)
    .mean()
)

# 168-hour rolling average (previous week's average)
df['rolling_168_mean'] = (
    df['Electricity consumption (MWh)']
    .shift(1)
    .rolling(168)
    .mean()
)

In [12]:
# Drop rows with NaN values caused by lag and rolling features
df = df.dropna().reset_index(drop=True)

df.shape

(52798, 14)

In [13]:
# Drop redundant End time column
df = df.drop(columns=['End time UTC'])

df.head()

,Start time UTC,Electricity consumption (MWh),hour,day_of_week,month,year,quarter,is_weekend,lag_1,lag_24,lag_168,rolling_24_mean,rolling_168_mean
0,2016-01-07 21:00:00,14074,21,3,1,2016,1,0,14303.0,13672.0,10800.0,14266.166667,12246.821429
1,2016-01-07 22:00:00,13643,22,3,1,2016,1,0,14074.0,13285.0,10431.0,14282.916667,12266.309524
2,2016-01-07 23:00:00,13369,23,3,1,2016,1,0,13643.0,13011.0,10005.0,14297.833333,12285.428571
3,2016-01-08 00:00:00,13142,0,4,1,2016,1,0,13369.0,12821.0,9722.0,14312.750000,12305.452381
4,2016-01-08 01:00:00,13111,1,4,1,2016,1,0,13142.0,12821.0,9599.0,14326.125000,12325.809524


In [14]:
# Define target variable (Predictor Variable)
y = df['Electricity consumption (MWh)']

# Drop columns not used as features
X = df.drop(columns=['Electricity consumption (MWh)', 'Start time UTC'])

X.head()

,hour,day_of_week,month,year,quarter,is_weekend,lag_1,lag_24,lag_168,rolling_24_mean,rolling_168_mean
0,21,3,1,2016,1,0,14303.0,13672.0,10800.0,14266.166667,12246.821429
1,22,3,1,2016,1,0,14074.0,13285.0,10431.0,14282.916667,12266.309524
2,23,3,1,2016,1,0,13643.0,13011.0,10005.0,14297.833333,12285.428571
3,0,4,1,2016,1,0,13369.0,12821.0,9722.0,14312.750000,12305.452381
4,1,4,1,2016,1,0,13142.0,12821.0,9599.0,14326.125000,12325.809524


In [15]:
 # 4. Time-series train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    shuffle=False   # Shuffle prevents random mixing and retains time series
)

print("Training size:", X_train.shape)
print("Testing size:", X_test.shape)

Training size: (42238, 11)
Testing size: (10560, 11)


In [16]:
# 5. Model Selection
# Now we move on to implementing the model we chose i.e XGBoost

# Initialize the model with reasonable default parameters
model = XGBRegressor(
    n_estimators=300,      # number of trees
    learning_rate=0.05,    # step size
    max_depth=6,           # tree depth
    subsample=0.8,         # row sampling
    colsample_bytree=0.8,  # feature sampling
    random_state=42
)

In [17]:
# Train model on training data
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

In [18]:
# Predict on test set
y_pred = model.predict(X_test)

In [19]:
# 6. Evaluation Metrics

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

MAE: 82.05
RMSE: 113.65
MAPE: 0.85%


In [20]:
# 7. Insights and Visualizations

import plotly.graph_objects as go

# Create figure
fig = go.Figure()

# Actual values
fig.add_trace(go.Scatter(
    x=df['Start time UTC'].iloc[-len(y_test):],
    y=y_test,
    mode='lines',
    name='Actual'
))

# Predicted values
fig.add_trace(go.Scatter(
    x=df['Start time UTC'].iloc[-len(y_test):],
    y=y_pred,
    mode='lines',
    name='Predicted'
))

fig.update_layout(
    title='Actual vs Predicted Electricity Consumption',
    xaxis_title='Time',
    yaxis_title='Electricity Consumption (MWh)'
)

fig.show()

In [21]:
# 8. Feature importance
importance = model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

feature_importance_df

,Feature,Importance
6,lag_1,0.605935
7,lag_24,0.288528
9,rolling_24_mean,0.036326
8,lag_168,0.029501
1,day_of_week,0.017654
0,hour,0.009669
5,is_weekend,0.007456
10,rolling_168_mean,0.001651
2,month,0.001647
4,quarter,0.001038


In [22]:
fig = px.bar(
    feature_importance_df,
    x='Importance',
    y='Feature',
    orientation='h',
    title='Feature Importance'
)

fig.show()

In [23]:
# 9. Error Distribution

error = y_test - y_pred

import plotly.express as px

fig = px.histogram(
    error,
    nbins=50,
    title='Prediction Error Distribution'
)

fig.show()

In [24]:
# 10. Baseline Benchmark

# Naive baseline prediction using lag_24 i.e Same hour yesterday
y_pred_baseline = X_test['lag_24']

# Evaluate baseline
mae_base = mean_absolute_error(y_test, y_pred_baseline)
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
mape_base = np.mean(np.abs((y_test - y_pred_baseline) / y_test)) * 100

print("Baseline Performance:")
print(f"MAE: {mae_base:.2f}")
print(f"RMSE: {rmse_base:.2f}")
print(f"MAPE: {mape_base:.2f}%")

Baseline Performance:
MAE: 366.78
RMSE: 507.48
MAPE: 3.73%


In [25]:
# 11. Tuning the model

model_tuned = XGBRegressor(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model_tuned.fit(X_train, y_train)

y_pred_tuned = model_tuned.predict(X_test)

In [26]:
# 12. Final Model  Evaluation Metrics

mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
mape_tuned = np.mean(np.abs((y_test - y_pred_tuned) / y_test)) * 100

print("Tuned Model Performance:")
print(f"MAE: {mae_tuned:.2f}")
print(f"RMSE: {rmse_tuned:.2f}")
print(f"MAPE: {mape_tuned:.2f}%")

Tuned Model Performance:
MAE: 79.78
RMSE: 112.35
MAPE: 0.82%
